In [15]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

#fonction qui crée une df du game log d'un gardien
def logGardien(idNHL):
    jsonJoueur = requests.get("https://statsapi.web.nhl.com/api/v1/people/" + idNHL + "/stats?stats=gameLog&season=20202021").json()
    df = pd.json_normalize(jsonJoueur["stats"][0]["splits"])
    gardienLogs = df[["season", "date", "isWin", "stat.shutouts"]]
    gardienLogs["gardien"] = idNHL
    return gardienLogs

#générer la liste de tous les gardiens de la table JOUEURS 
dfGardiens = pd.read_sql("SELECT idNHL FROM JOUEURS WHERE position == 'G'", conn)
gardiens = dfGardiens["idNHL"].apply(str)
listGardiens = gardiens.values.tolist()

#créer une df vide pour plus tard append le return de logGardien(idNHL) 
vraisCol = ["season", "date", "isWin", "stat.shutouts", "gardien"]
gameLogsG = pd.DataFrame(columns=vraisCol)
gameLogsG = gameLogsG.fillna(0)

#itérer logGardien(idNHL) à la liste des gardiens de la table JOUEURS et sauvegarder les résultats à gameLogsG
for i in listGardiens:
    try:
        gardienLogs = logGardien(i)
    except:
        print("fausse alarme (espoir) " + i)
    gameLogsG = gameLogsG.append(gardienLogs)
gameLogsG = gameLogsG.rename(columns = {"date" : "dateMatch"})
gameLogsG[["dateMatch"]] = gameLogsG.dateMatch.str.split(" 00:",expand=True)
gameLogsG["dateMatch"] = pd.to_datetime(gameLogsG["dateMatch"])
pbpDates = pd.read_sql("SELECT MAX(Date) FROM PBP", conn)
maxDate = pbpDates.iloc[0]["MAX(Date)"]
isInRange = maxDate

fausse alarme (espoir) 8482137
fausse alarme (espoir) 8481519


'2021-02-02'

In [5]:
# Faire le calcul de points avant de le mettre en SQL
a = gameLogsG["isWin"]
gameLogsG["esWin"] = a * 2
gameLogsG ["scorePool"] = gameLogsG["esWin"] + gameLogsG["stat.shutouts"]

# save la df des game logs en une table SQL
gameLogsG.to_sql("GARDIENS", conn, if_exists="replace", index=False)

# faire le update SQL
def GwriteUpdate(typeScore):
    colAl = ""
    if typeScore == "isWin":
        colAl = "buts"
    elif typeScore == "stat.shutouts":
        colAl = "assist"
    elif typeScore == "scorePool":
        colAl = "points"
    debut = "UPDATE ALIGNEMENTS SET " + colAl + "Actuels = (SELECT Nb FROM (SELECT gardien, SUM(" + typeScore + ") AS NbV FROM GARDIENS WHERE dateMatch BETWEEN ALIGNEMENTS.dateDebut AND ALIGNEMENTS.dateFin AND gardien = ALIGNEMENTS.idNHL)) WHERE ALIGNEMENTS.idNHL IN (SELECT DISTINCT(gardien) FROM GARDIENS)"
    return debut

NameError: name 'gameLogsG' is not defined

In [1]:
#la majG() ne doit pas être jusqu'à 2100-01-01 mais jusqu'à la date max, ou on s'arrange pour que le pbp soit tjrs à jour.

conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

def GwriteUpdate(typeScore):
    colAl = ""
    if typeScore == "isWin":
        colAl = "buts"
    elif typeScore == "'stat.shutouts'":
        colAl = "assist"
    elif typeScore == "scorePool":
        colAl = "points"
    debut = "UPDATE ALIGNEMENTS SET " + colAl + "Actuels = (SELECT Nb FROM (SELECT gardien, SUM(" + typeScore + ") AS Nb FROM GARDIENS WHERE dateMatch BETWEEN ALIGNEMENTS.dateDebut AND ALIGNEMENTS.dateFin AND gardien = ALIGNEMENTS.idNHL)) WHERE ALIGNEMENTS.idNHL IN (SELECT DISTINCT(gardien) FROM GARDIENS)"
    return debut

def majG():
    updVic = GwriteUpdate("isWin")
    updJB = GwriteUpdate("'stat.shutouts'")
    updSco = GwriteUpdate("scorePool")
    execSQL(updVic, conn, write = True)
    execSQL(updJB, conn, write = True)
    execSQL(updSco, conn, write = True)

majG()
